# **Instalação PySpark**

In [1]:
#check that java is installed
!java -version

openjdk version "11.0.27" 2025-04-15
OpenJDK Runtime Environment (build 11.0.27+6-post-Ubuntu-0ubuntu122.04)
OpenJDK 64-Bit Server VM (build 11.0.27+6-post-Ubuntu-0ubuntu122.04, mixed mode, sharing)


In [2]:
#install pyspark
!pip install pyspark

In [93]:
import os
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, LongType, TimestampType, DateType

In [4]:
#use curl to download the data then unzip it into the content directory
!curl -O https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_2023/raw/review_categories/Automotive.jsonl.gz
!gunzip -f Automotive.jsonl.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   284  100   284    0     0     20      0  0:00:14  0:00:13  0:00:01    71

gzip: Automotive.jsonl.gz: not in gzip format


In [5]:
print(f'The size of the Amazon Automotive Revie file is: {os.path.getsize("/content/Automotive.jsonl.gz") / (1024 ** 3):.2f} GB')

The size of the Amazon Automotive Revie file is: 0.00 GB


In [6]:
spark = SparkSession.builder.appName('AutomotiveData').getOrCreate()

print(f'The Spark version is {spark.version}')

The Spark version is 3.5.1


In [98]:
from pyspark.sql.functions import col
from pyspark.sql.functions import substring, date_format, to_date

# **ETL**

In [54]:
df = spark.read.csv('/content/car_prices.csv', header=True, inferSchema=True)
df.show(10)

+----+---------+-------------------+--------------------+-----------+------------+-----------------+-----+---------+--------+-----+--------+--------------------+-----+------------+--------------------+
|year|     make|              model|                trim|       body|transmission|              vin|state|condition|odometer|color|interior|              seller|  mmr|sellingprice|            saledate|
+----+---------+-------------------+--------------------+-----------+------------+-----------------+-----+---------+--------+-----+--------+--------------------+-----+------------+--------------------+
|2015|      Kia|            Sorento|                  LX|        SUV|   automatic|5xyktca69fg566472|   ca|        5|   16639|white|   black|kia motors americ...|20500|       21500|Tue Dec 16 2014 1...|
|2015|      Kia|            Sorento|                  LX|        SUV|   automatic|5xyktca69fg561319|   ca|        5|    9393|white|   beige|kia motors americ...|20800|       21500|Tue Dec 16 2

In [13]:
df.printSchema()

root
 |-- year: integer (nullable = true)
 |-- make: string (nullable = true)
 |-- model: string (nullable = true)
 |-- trim: string (nullable = true)
 |-- body: string (nullable = true)
 |-- transmission: string (nullable = true)
 |-- vin: string (nullable = true)
 |-- state: string (nullable = true)
 |-- condition: integer (nullable = true)
 |-- odometer: integer (nullable = true)
 |-- color: string (nullable = true)
 |-- interior: string (nullable = true)
 |-- seller: string (nullable = true)
 |-- mmr: integer (nullable = true)
 |-- sellingprice: integer (nullable = true)
 |-- saledate: string (nullable = true)



In [40]:
for coluna in df.columns:
  print(coluna, df.filter(df[coluna].isNull()).count())

year 0
make 0
model 0
trim 316
body 2492
transmission 0
vin 4
state 0
condition 9866
odometer 59
color 581
interior 581
seller 0
mmr 26
sellingprice 0
saledate 0


In [55]:
df = df.filter(col('model').isNotNull() & col('sellingprice').isNotNull() & col('transmission').isNotNull())

In [56]:
df = df.select(col('year'),col('make'), col('model'), col('transmission'),\
          col('state'),col('seller'), col('sellingprice'),col('saledate'))
df.show(10)

+----+---------+-------------------+------------+-----+--------------------+------------+--------------------+
|year|     make|              model|transmission|state|              seller|sellingprice|            saledate|
+----+---------+-------------------+------------+-----+--------------------+------------+--------------------+
|2015|      Kia|            Sorento|   automatic|   ca|kia motors americ...|       21500|Tue Dec 16 2014 1...|
|2015|      Kia|            Sorento|   automatic|   ca|kia motors americ...|       21500|Tue Dec 16 2014 1...|
|2014|      BMW|           3 Series|   automatic|   ca|financial service...|       30000|Thu Jan 15 2015 0...|
|2015|    Volvo|                S60|   automatic|   ca|volvo na rep/worl...|       27750|Thu Jan 29 2015 0...|
|2014|      BMW|6 Series Gran Coupe|   automatic|   ca|financial service...|       67000|Thu Dec 18 2014 1...|
|2015|   Nissan|             Altima|   automatic|   ca|enterprise vehicl...|       10900|Tue Dec 30 2014 1...|
|

In [66]:
df2 = df

In [77]:
df2 = df2.withColumn('date', substring('saledate', 1,15))
df2.show(10)

+----+---------+-------------------+------------+-----+--------------------+------------+--------------------+---------------+
|year|     make|              model|transmission|state|              seller|sellingprice|            saledate|           date|
+----+---------+-------------------+------------+-----+--------------------+------------+--------------------+---------------+
|2015|      Kia|            Sorento|   automatic|   ca|kia motors americ...|       21500|Tue Dec 16 2014 1...|Tue Dec 16 2014|
|2015|      Kia|            Sorento|   automatic|   ca|kia motors americ...|       21500|Tue Dec 16 2014 1...|Tue Dec 16 2014|
|2014|      BMW|           3 Series|   automatic|   ca|financial service...|       30000|Thu Jan 15 2015 0...|Thu Jan 15 2015|
|2015|    Volvo|                S60|   automatic|   ca|volvo na rep/worl...|       27750|Thu Jan 29 2015 0...|Thu Jan 29 2015|
|2014|      BMW|6 Series Gran Coupe|   automatic|   ca|financial service...|       67000|Thu Dec 18 2014 1...|T

In [88]:
df = df2.withColumn("date_sale",date_format(to_date(col("date"), "EEE MMM dd yyyy"), "yyyy-MM-dd"))

In [ ]:
df = df.withColumn('date_sale', col('date_sale').cast(DateType()))
df.show(5)

In [89]:
df = df.select(col('year'),col('make'), col('model'), col('transmission'),\
          col('state'),col('seller'), col('sellingprice'),col('date_sale'))
df.show(10)

+----+---------+-------------------+------------+-----+--------------------+------------+----------+
|year|     make|              model|transmission|state|              seller|sellingprice| date_sale|
+----+---------+-------------------+------------+-----+--------------------+------------+----------+
|2015|      Kia|            Sorento|   automatic|   ca|kia motors americ...|       21500|2014-12-16|
|2015|      Kia|            Sorento|   automatic|   ca|kia motors americ...|       21500|2014-12-16|
|2014|      BMW|           3 Series|   automatic|   ca|financial service...|       30000|2015-01-15|
|2015|    Volvo|                S60|   automatic|   ca|volvo na rep/worl...|       27750|2015-01-29|
|2014|      BMW|6 Series Gran Coupe|   automatic|   ca|financial service...|       67000|2014-12-18|
|2015|   Nissan|             Altima|   automatic|   ca|enterprise vehicl...|       10900|2014-12-30|
|2014|      BMW|                 M5|   automatic|   ca|the hertz corpora...|       65000|20

In [96]:
df.printSchema()

root
 |-- year: integer (nullable = true)
 |-- make: string (nullable = true)
 |-- model: string (nullable = true)
 |-- transmission: string (nullable = true)
 |-- state: string (nullable = true)
 |-- seller: string (nullable = true)
 |-- sellingprice: integer (nullable = true)
 |-- date_sale: date (nullable = true)



In [97]:
df = df.withColumnRenamed('year', 'ano').withColumnRenamed('make', 'fabricante')\
  .withColumnRenamed('model', 'modelo').withColumnRenamed('transmission', 'cambio')\
  .withColumnRenamed('state', 'estado').withColumnRenamed('seller', 'vendedor')\
  .withColumnRenamed('sellingprice', 'preco_venda').withColumnRenamed('date_sale', 'data_venda')

df.show(20)

+----+----------+-------------------+---------+------+--------------------+-----------+----------+
| ano|fabricante|             modelo|   cambio|estado|            vendedor|preco_venda|data_venda|
+----+----------+-------------------+---------+------+--------------------+-----------+----------+
|2015|       Kia|            Sorento|automatic|    ca|kia motors americ...|      21500|2014-12-16|
|2015|       Kia|            Sorento|automatic|    ca|kia motors americ...|      21500|2014-12-16|
|2014|       BMW|           3 Series|automatic|    ca|financial service...|      30000|2015-01-15|
|2015|     Volvo|                S60|automatic|    ca|volvo na rep/worl...|      27750|2015-01-29|
|2014|       BMW|6 Series Gran Coupe|automatic|    ca|financial service...|      67000|2014-12-18|
|2015|    Nissan|             Altima|automatic|    ca|enterprise vehicl...|      10900|2014-12-30|
|2014|       BMW|                 M5|automatic|    ca|the hertz corpora...|      65000|2014-12-17|
|2014| Che

In [99]:
df.write \
    .option("header", True) \
    .option("delimiter", ",") \
    .mode("overwrite") \
    .csv("/content/Carros.csv")